In [1]:
import os
import time
import yaml
import argparse
import pandas as pd

import orca
import autospec_recipes
from urbansim.utils import misc

parcel
nrpm_constraints.yaml
elcm_constraints.yaml
hlcm_constraints.yaml
__init__.py
rpm_constraints.yaml
rdplcm_constraints.yaml
nrdplcm_constraints.yaml
block
elcm_constraints.yaml
hlcm_constraints.yaml
__init__.py
repm_value_constraints.yaml
rdplcm_constraints.yaml
repm_rent_constraints.yaml
semcog
elcm_constraints.yaml
hlcm_constraints.yaml
__init__.py
elcmhb_constraints.yaml
zone
elcm_constraints.yaml
repm_nonres_constraints.yaml
hlcm_constraints.yaml
__init__.py
repm_value_constraints.yaml
rdplcm_constraints.yaml
repm_rent_constraints.yaml
nrdplcm_constraints.yaml


In [2]:
def register_config_injectable_from_yaml(injectable_name, yaml_file):
    @orca.injectable(injectable_name, cache=True)
    def func():
        with open(os.path.join(misc.configs_dir(), yaml_file)) as f:
            config = yaml.load(f)
            return config
    return func

for yaml_file in ["settings.yaml"]:
    injectable_name = yaml_file.split('.')[0]
    register_config_injectable_from_yaml(injectable_name, yaml_file)

settings = orca.get_injectable('settings')

orca_modules = settings['orca_modules']
pre_processing_steps = settings['orca_preprocessing_steps']
simulation_steps = settings['orca_simulation_steps']

for module in orca_modules:
    exec 'import %s' % module

In [4]:
if len(pre_processing_steps) > 0:
     orca.run(pre_processing_steps)

Running step 'build_networks'
Time to execute step 'build_networks': 24.80 s
Running step 'neighborhood_vars'
Computing accessibility variables
Computing sum_residential_units
Removed 4 rows because they contain missing values
Computing sum_nonresidential_units
Removed 4 rows because they contain missing values
Computing ave_unit_sqft
Computing ave_lot_sqft
Removed 4 rows because they contain missing values
Computing population
Computing households
Computing midinc_hhs
Computing lowinc_hhs
Computing highinc_hhs
Computing race_1_hhs
Computing race_2_hhs
Computing race_3_hhs
Computing race_4_hhs
Computing hhs_with_children
Computing sfdu
Computing hhsize
Computing ave_income
Computing max_industrial_far
Computing max_office_far
Computing max_retail_far
Computing max_medical_far
Computing jobs
Removed 6 rows because they contain missing values
Computing retail_jobs
Computing node_r1500_sqft
Computing residential
Computing ave_nonres_sqft_price
Removed 1 rows because they contain missing v

In [5]:
b = orca.get_table('buildings').to_frame()

Disaggregating sum_residential_units to buildings from nodes_walk
Disaggregating percent_low_income to buildings from nodes_walk
Disaggregating households to buildings from nodes_walk
Disaggregating ave_lot_sqft to buildings from nodes_walk
Disaggregating z_total_jobs to buildings from zones
Disaggregating max_office_far to buildings from nodes_walk
Disaggregating race_3_hhs to buildings from nodes_walk
Disaggregating percent_mid_income to buildings from nodes_walk
Disaggregating max_far to buildings from parcels
Disaggregating total_job_spaces to buildings from parcels
Disaggregating employment to buildings from zones
Disaggregating lowinc_hhs to buildings from nodes_walk
Disaggregating county_id to buildings from parcels
Disaggregating drv_45min_jobs to buildings from nodes_drv
Disaggregating sfdu to buildings from nodes_walk
Disaggregating pct_undev to buildings from parcels
Disaggregating node_r1500_sqft to buildings from nodes_walk
Disaggregating medical to buildings from nodes_wa

In [6]:
nulls = b.isnull().sum()
for variable in nulls.index.values:
    if nulls[variable] > 0:
        print variable
        if variable not in ['city_id', 'nodeid_drv', 'zone_id', 'nodeid_walk', 'large_area_id']:
            b[variable][b[variable].isnull()] = b[variable].mean()
        else:
            b[variable][b[variable].isnull()] = 0

city_id


/home/janowicz/anaconda2/envs/mtc2/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/janowicz/anaconda2/envs/mtc2/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


drv_nearest_library
semmcd
nodeid_drv
drv_nearest_healthcenter
drv_nearest_park
walk_nearest_grocery
zone_id
walk_nearest_healthcenter
drv_nearest_grocery
y
school_district_achievement
hedonic_sector_id
x
walk_nearest_hospital
drv_nearest_hospital
nodeid_walk
walk_nearest_park
parcel_sqft
large_area_id
walk_nearest_urgentcare
drv_nearest_urgentcare
walk_nearest_library


In [7]:
for config in orca.get_injectable('constraint_configs')['semcog']['hlcm_constraints.yaml']:
    for varname in config['variables']:
        if varname not in b.columns:
            print varname

ln_income:building_type_id_is_81
ln_income:parcels_acres
ln_income:nodes_walk_ave_income
ln_income:sqft_per_unit
ln_income:nodes_walk_ln_popden
ln_income:sqft_price_res
persons:nodes_walk_ln_popden
persons:sqft_per_unit
hhsize_gt_2:building_type_id_is_81
has_children:building_type_id_is_81
has_children:sqft_per_unit
has_children:nodes_walk_percent_hh_with_children
has_children:nodes_drv_elem_school_perf
is_young:zones_empden
is_young:nodes_walk_retail_jobs
has_workers:nodes_drv_drv_45min_jobs
has_workers:zones_a_ln_emp_50min_transit
is_race1:nodes_walk_percent_race1
is_race2:nodes_walk_percent_race2
is_race3:nodes_walk_percent_race3
is_race4:nodes_walk_percent_race4


In [8]:
for config in orca.get_injectable('constraint_configs')['semcog']['elcm_constraints.yaml']:
    for varname in config['variables']:
        if varname not in b.columns:
            print varname

crime08


In [9]:
j = orca.get_table('jobs').to_frame()
j['slid'] = (j.sector_id*100000 + j.large_area_id).fillna(0).astype('int')

In [10]:
h = orca.get_table('households').to_frame()

variables/variables_demographic.py:76: RuntimeWarning: invalid value encountered in log1p
  return np.log1p(households.income)


In [14]:
h['qlid'] = (h.income_quartile*100000 + h.large_area_id).fillna(0).astype('int')

In [17]:
with pd.HDFStore('expvars.h5', mode='a', complib='zlib', complevel=1) as store:
    store['buildings'] = b
    store['jobs'] = j
    store['households'] = h

In [19]:
h.qlid.value_counts().index.values

array([400125, 100005, 300125, 300003, 200125, 200003, 400003, 100003,
       300099, 200099, 100125, 400099, 100099, 200005, 400161, 300005,
       300161, 200161, 100161, 400093, 300093, 200147, 300147, 300115,
       400005, 100147, 200115, 200093, 400115, 400147, 100115, 100093])